import libraries

In [3]:
import numpy
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from pathlib import Path
import math
from math import pi
from math import sqrt
import statistics
import scipy.integrate
from scipy.signal import find_peaks
from scipy.signal import argrelmin
from numpy import trapz
from scipy.integrate import cumtrapz

user inputs

In [4]:
# SETUP VARIABLES - USER INPUTS
BD = 2 #Bluedrop file is from 
soiltype = "s" #s = sand, c=clay, m=mixed, u=unknown
atype = 'm'  # m = mantle area (best for sands), p = projected area (best for clays)
tiptype = 'c'  # c = cone, p = parabolic, b = blunt
#offset = 1 # this value is subtracted from the accelerometer readings
droptype = 'a' #w = water, #a = air
sign = "uk" #enter an effective unit weight value in kg/m^3 or "uk" if unknown 
# paste the filepath to the folder where the BD data is stored
binFilepath = Path("H:/.shortcut-targets-by-id/1aF9t2aiRGWTftJMZFAOBixqvQniFBjnb/Duck  2023/Data/Intertidal/BlueDrop, Samples & Moisture Gage/08March23/BD2 3.8.23 - Drops Only")
#paste the filepath to an excel file that the analysis results will be printed in

set functions

In [5]:

# FUNCTIONS FOR USE IN ANALYSIS
def masslength(tiptype): #sets the mass and length of the pentrometer based on the tip
    global mass
    global length
    if tiptype == 'c':
        mass = 7.71 #kg
        length = 8.48-0.93 #originally 7.87, 7.57 for perfect 60 deg, 8.48 measured - .93 measured 90 deg
    elif tiptype == 'p':
        mass = 9.15
        length = 8.26
    elif tiptype == 'b':
        mass = 10.30
        length = 8.57

def dropstartend(peak): #after locating the peaks, this function chops the minute long file into a smaller segment immediately before and after the peak
    global dropstart
    global dropend
    if peak <= 1500:
        dropstart = 1
        dropend = peak + 500
    elif peak > 119500:
        dropstart = peak - 1500
        dropend = 120000
    else:
        dropstart = peak - 1500
        dropend = peak + 500

def accPick(dg, d): #this function picks the smallest accelerometer that's not maxed out to perform the integration on
    maxAcc = dg["250g (g)"].max()
    global acc
    global accName
    global accNameg
    global accg
    if maxAcc < 5: #- offset:
        if dg["2g (g)"].max() < 1.8: #offset:  # does an extra check for the 2g because of noise
            acc = dfCal["2g (m/s^2)"]
            accg = dfCalg["2g (g)"]
            accName = "2g (m/s^2)"
            accNameg = "2g (g)"
        else:
            acc = dfCal["18g (m/s^2)"]
            accg = dfCalg["18g (g)"]
            accName = "18g (m/s^2)"
            accNameg = "18g (g)"
    elif maxAcc < 18: #- offset
        acc = dfCal["18g (m/s^2)"]
        accg = dfCalg["18g (g)"]
        accName = "18g (m/s^2)"
        accNameg = "18g (g)"
    elif maxAcc < 50: #- offset:
        acc = dfCal["50g (m/s^2)"]
        accg = dfCalg["50g (g)"]
        accName = "50g (m/s^2)"
        accNameg = "50g (g)"
    else:
        acc = dfCal["250g (m/s^2)"]
        accg = dfCalg["50g (g)"]
        accName = "250g (m/s^2)"
        accNameg = "250g (g)"

def findchangepts(): #This function picks the moment that the BD impacts the ground (not in use currently)
    global drop
    jlist = list()
    global jindex
    print("finding start of drop...")
    for i in range(4,len(accg)-4):
        p1 = 1
        #print(p1)
        p2 = i
        #print(p2)
        p3 = len(accg)
        #print(p3)
        sample1 = list(accg[p1:p2-1])
        #print(sample1)
        sample2 = list(accg[p2:p3])
        #print(sample2)
        stat1 = math.log(statistics.variance(sample1))
        stat2 = math.log(statistics.variance(sample2))
        #print(stat1)

        j1 = (i-1)*stat1
        j2 = ((len(accg)-1)-i+1)*stat2
        j = j1+j2
        #print(j)
        jlist.append(j)

    drop = min(jlist)
    #print("drop is")
    #print(drop)
    jlist = np.array(jlist)
    #print(jlist)
    #print(jlist.size)
    jlist = np.insert(jlist, 0, (0,0,0,0)) #reshape to match up with dataframe d
    jlist = np.append(jlist, (0,0,0,0)) #reshape to match up with dataframe d
    #print(jlist.size) #should be 2000
    jindex = np.where(jlist==drop) #finds the index of the drop start
    jindex = int(jindex[0]) #converts the index into a number from a tuple
    print("Jlist: ", jlist)

def finddropend(n): #finds the location after the peak where the deceleration is 1 
    global penend
    below0list = list()
    #for i in range(dropstart+jindex, dropend, 1):
    for i in range(peak, dropend, 1):
        if accg[i] < 1: #the offset is applied later in the code
            penend = i 
            below0list = np.append(below0list, penend)
    penend=int(min(below0list))

def integration(d, accint): #integrates the deceleration data to solve for velocity and penetration depth
    global vel
    global maxvel
    global dep
    global maxdep
    #accint = acc[penstart:penend]
    #print(len(d))
    #print(len(accint))
    vel = scipy.integrate.cumtrapz(accint, x=d["Time (s)"])  # solves for velocity
    vel = np.array(vel)
    vel = numpy.insert(vel, 0, 0) #not sure what to insert here, but it makes it the right size
    vel = np.flip(vel)
    maxvel = vel.max()
    maxvel = round(maxvel,1)
    dep = scipy.integrate.cumtrapz(vel, x=d["Time (s)"])  # solves for penetration depth
    dep = numpy.insert(dep, 0, 0)  # not sure what to insert here, but it makes it the right size
    maxdep = dep.max()
    maxdep = round(maxdep,4)
    d.insert(9, "Velocity (m/s)", vel)
    d.insert(10, "Penetration Depth (m)", dep)

def peakpick():
    global penstart
    fig, (ax1) = plt.subplots(1)
    plt.plot(dfCalg['Count'], dfCalg[accNameg], label=accNameg)
    plt.plot(dfCalg['Count'], dfCalg['2g (g)'], label='2g')

    ax1.set_xlim(left=dropstart, right=dropend)
    ax1.legend()
    ax1.set(ylabel="Deceleration (g)")
    ax1.set(xlabel="Time")
    ax1.set_title('Double-click start of drop #' + str(a))
    startendpt = plt.ginput(2, 0)
    pentimes = []
    for t in startendpt:
        for x in t:
            pentimes.append(x)

    penstart = int(pentimes[0])
    print("start of penetration: ", penstart)

def CheckingFunction(): # (to be used with auto peak finding) This function is added to check the values of the start and end of the drop if the autofinding function doesn't work

    global jindex
    global num1
    
    xvalues=np.linspace(0,len(dg)-1,len(dg))
    plt.figure(num=1)
    temp_dg=dg['250g (g)']
    plt.plot(xvalues,temp_dg)
    plt.grid(visible=True,which='both')
    plt.show()
    print ('The start of the drop is', jindex)
    print ('The end of the drop is', num1)
    print('Are the start and end of the drop correct?(y/n)')
    drop_input=input()
    if drop_input=='n':   
        peakpick()
        jindex = penstart
        num1 = penend

def areafind(): #finds the embedded area based on the penetration depth, area type, and the tip
    global area
    global trunc_index
    a1 = list() #placeholder for the penetrated area at that specific depth
    d1 = dep*100 #penetration depth array, in cm
    #print(len(d1))
    r = list() #placeholder for the radius at that specific depth
    if tiptype == 'c':
        if atype == 'm':
            for k in range(0,len(d1)):
                if d1[k]<length:
                    r.append(.22+d1[k]*((4.375-.22)/7.55))
                    #r.append(d1[k]*np.tan(30*pi/180)) this is from the original matlabcode
                    a1.append(pi*r[k]*(sqrt((r[k]**2)+(d1[k]**2))))
                elif d1[k]>=length:
                    r.append(4.375)
                    a1.append(pi*r[k]*(sqrt((r[k]**2)+(length**2))))
                a1[k] = a1[k]/10000
            area = a1
            #print(r)
            print(np.column_stack((d1,r, area)))
        elif atype == 'p':
            for k in range(0,len(d1)):
                if d1[k]<length:
                    r.append(.22+d1[k]*((4.375-.22)/7.55))
                    # r.append(d1[k]*np.tan(30*pi/180))
                    a1.append(pi*r[k]**2)
                elif d1[k]>=length:
                    r.append(4.375)
                    if r[k-1]<r[k]:
                        trunc_index = r.index(r[k-1])+jindex+dropstart #performs analysis only on readings beyond depth of cone height
                    a1.append(pi*(r[k])**2)
                a1[k] = a1[k]/10000
            area = a1

    elif tiptype == 'b':
        if atype =='m':
            for k in range(0,len(d1)):
                if d1[k]<length:
                    r.append(4.375)
                    a1.append(pi*r[k]**2 + 2*pi*r[k]*d1[k])
                if d1(k)>=length:
                    r.append(4.375)
                    a1.append(pi*r[k]**2 + 2*pi*r[k]*length)
                a1[k]=a1[k]/10000
            area = a1
        elif atype == 'p':
            for k in range(0,len(d1)):
                a1.append(pi*4.375**2)
                a1[k]=a1[k]/10000
            area = a1

    '''elif tiptype == "p":
        if atype == 'm':
            for k in range(1,len(d)):
                if d1[k]<length:
                     r[k]=sqrt(2.4184*d1[k])
                           polarfun = @(theta,r) r.*sqrt(0.745*r.^2 + 1);
                           A1(k)= integral2(polarfun,0,2*pi,0,r(k));
            
        elif atype == 'p':
            for k in range(1,len(d)):'''

def qdynfun(acc): # calculates dynamic bearing capacity
    global qdyn
    global maxbcdep
    global maxqdyn
    global qdyntable

    buoy = 1020*0.002473
    if droptype == "w": #water drops
        Fbe = (mass-buoy)*acc #drop force

    elif droptype =="a": #air drops
        Fbe = mass*acc

    qdyn = (Fbe/area)/1000 #Dynamic bearing capacity (kPa)
    qdyntable = pd.DataFrame()
    qdyntable["Qdyn"] = qdyn
    qdyntable["Depth"] = dep
    qdyntablet = qdyntable#.truncate(before=trunc_index)
    qdynt = qdyntablet["Qdyn"]
    #print(qdyntable)

    maxqdyn = qdynt.max() #gives maximum qdyn
    maxbcdep = qdyntablet.loc[qdyntable["Qdyn"]==maxqdyn, 'Depth'].item()

    maxqdyn = round(maxqdyn,1)
    maxbcdep = round(maxbcdep,4)
    #print("maxbcdep = ", maxbcdep) #finds the penetration depth corresponding to the maximum qdyn


    """bcmin = argrelmin(qdyn)
    peaksArray = np.array(bcmin)
    print("Peaks Array = ", peaksArray)"""

def qsbcfun(acc): #calculates quasi-static bearing capacity
    global bctable
    global qsbc
    global maxHqsbc
    global maxLqsbc
    global maxAqsbc

    if droptype == "w":
        srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction. 
        srfk = [0.2, 0.4, 1, 1.5] #list of strain rate factors to run; must be in increasing order

        bctable = pd.DataFrame()
        for i in range(0,len(srfk)):
            fsr = 1+srfk[i]*srcv
            qsbc = qdyn/fsr
            bctable.insert(i, "qsbc for srf = "+str(srfk[i]), qsbc)

        bctable_avgs = pd.DataFrame()
        bctable_avgs["High Average qsbc"]= (bctable.iloc[:,0]+bctable.iloc[:,1])/2 #Average of the lowest 2 strain rate factors
        bctable_avgs["Low Average qsbc"] = (bctable.iloc[:,int(len(srfk))-1]+bctable.iloc[:,int(len(srfk)-2)])/2 #average of the highest 2 strain rate factors
        maxHqsbc = bctable_avgs["High Average qsbc"].max()
        maxHqsbc = round(maxHqsbc, 1)
        maxLqsbc = bctable_avgs["Low Average qsbc"].max()
        maxLqsbc = round(maxLqsbc,1)
        maxAqsbc = "-"

        #print(bctable)

    elif droptype == "a":
        srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction. 
        srfk = 0.31*maxvel
        fsr = 1+srfk*srcv
        qsbc = qdyn/fsr
        maxHqsbc = "-"
        maxLqsbc = "-"
        maxAqsbc = qsbc.max()
        maxAqsbc = round(maxAqsbc,1)

def dr(accg): #Albatal 2019 method for calculating relative density (Duck, NC specific!)
    global Dr
    maxacc = accg.max() #maximum deceleration, in g
    #print("Max deceleration = ", maxacc)
    Dr = -2.18*10**-4*maxacc**3+1.29*10**-2*maxacc**2+1.61*maxacc-13.09 #Albatal 2019
    Dr = round(Dr, 1)
    #print("Relative density is ", dr)

def ignore_drop():
    global keep_drop
    print('Keep Drop? (y/n)')
    keep_drop=input()

def duncan_correlation(): #duncan correlation for caluclating friction angle
    global phi
    global sign
    #coefficients A, B, C, and D used in the Duncan Correlation. Duncan[34,10,3,2], Albatal[34,10,2,5]
    A = 34
    B = 10
    C = 3
    D = 2
    pa = 101.325 #kPa
    print("max penetration depth = ", maxdep)
    if sign == "uk":
        sign = 1120*maxdep #1120 = 70pcf in kg/m^3

    phi = A + B*Dr/100-(C+D*Dr/100)*math.log(sign/pa)
    phi = round(phi, 1)
    print("phi =", phi)

def firmnessfactor(acc, drop):
    global maxacc
    global ff
    maxacc = acc.max()
    g = 9.80665
    tp = len(drop)/2000

    ff = maxacc/(g*tp*maxvel)
    ff = round(ff,1)
    print("Firmness factor = ", ff)

def bctruncate(dropg, drop, acc, qdyn):
    dropg = pd.DataFrame(dropg)
    drop = pd.DataFrame(drop)
    qdyn = pd.DataFrame(qdyn)
    acc = pd.DataFrame(acc)
    dropg = dropg.truncate(before=trunc_index)
    drop = drop.truncate(before=trunc_index)
    acc = acc.truncate(before=trunc_index)
    qdyn = qdyn.truncate(before=trunc_index)


In [8]:
#Plots for exporting
def overviewplot(): #Plot showing all accellerometers and pore pressure readings
    fig, (ax1, ax2, ax3) = plt.subplots(3)
    fig.set_size_inches(14, 7)
    #plt.tight_layout()
    ax1.plot(time, g250g, label="250g")
    ax1.plot(time, g50g, label="50g")
    ax1.plot(time, g18g, label="18g")
    ax1.plot(time, g2g, label="2g")
    ax1.legend(loc = "upper right")
    ax1.set(ylabel="Deceleration (g)")
    ax1.set_title("BD file "+fileNum)
    ax1.grid()

    ax2.plot(time, ppm, label="Pore Pressure")
    ax2.set(ylabel="Pore Pressure (kPa)")
    ax2.grid()

    ax3.plot(time, gX55g, label="X 55g")
    ax3.plot(time, gY55g, label="Y 55g")
    ax3.legend(loc = "upper right")
    ax3.set(ylabel="Deceleration (g)")
    ax3.set(xlabel="Time (s)")
    ax3.grid()

    fig.subplots_adjust(bottom=.1, left = .1)
    plotName = fileNum+" Overview.png"
    #plt.savefig(plotPath / plotName)
    plt.show()

def integplot(drop, accName): #Deceleration,Velocity,and penetration depth vs time plots
    fig, (ax1, ax2, ax3) = plt.subplots(3)
    fig.set_size_inches(7, 7)
    #resize to 7,7, 7,14
    #plt.tight_layout()
    timereset = drop["Time (s)"]- drop["Time (s)"].iloc[0]
    ax1.plot(timereset, drop[accName], color = "k" ) #,marker = 11
    ax1.set(ylabel="Deceleration (m/s^2)")
    ax1.set_title("BD file "+fileNum+ " Drop #"+str(a))
    ax1.grid()

    ax2.plot(timereset, drop['Velocity (m/s)'] , color = "k" )
    ax2.set(ylabel="Velocity (m/s)")
    ax2.grid()

    ax3.plot(timereset, drop["Penetration Depth (m)"] , color = "k")
    ax3.set(ylabel="Penetration Depth (m)", xlabel="Time(s)")
    ax3.grid()

    fig.subplots_adjust(bottom=.1, left = .1)
    plotName = fileNum+" "+str(a)+" Integ"
    #plt.savefig(plotPath / plotName)
    plt.show()

def dep_qsbc_comboplot(dropg, drop, accNameg, qdyn):
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(7, 7)
    ax1.plot(dropg[accNameg], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "solid") #marker = 11
    ax1.plot(drop["Velocity (m/s)"], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "dashed")
    ax1.plot(drop["X55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "b", linestyle = "dashed")
    ax1.plot(drop["Y55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "r", linestyle = "dashed")
    ax1.set(xlabel="Deceleration (g) and Velocity (m/s)", ylabel="Penetration Depth (cm)")
    ax1.invert_yaxis()
    ax1.legend(["Deceleration (g)", "Velocity (m/s)", "X tilt (g)", "Y tilt (g)"], loc = "upper right")
    ax1.set_title("BD file "+fileNum)
    ax1.grid()

    if droptype == "a": #for partial saturation, only one qsbc curve
        ax2.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax2.plot(qsbc, drop["Penetration Depth (m)"]*100, label="QSBC") #plots QSBC
        ax2.set(xlabel="Bearing Capacity (kPa)")
        ax2.set_xlim(0,800)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", "QSBC"], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()

    else: #for saturated drops, use a range of strain-rate factors
        ax2.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax2.plot(bctable.iloc[:,0], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[0])) #plots QSBC with smallest srf
        ax2.plot(bctable.iloc[:,len(bctable.columns)-1], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[len(bctable.columns)-1])) #plots QSBC with largest srf
        for i in range(1,len(bctable.columns)-1): #plots qsbc for all other srfs
            ax2.plot(bctable.iloc[:,i], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[i]), color = '.6')
        ax2.set(xlabel="Bearing Capacity (kPa)")
        ax2.set_xlim(0,)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", str(bctable.columns[0]), str(bctable.columns[len(bctable.columns)-1])], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()
    
    fig.subplots_adjust(bottom=.1, left = .1)
    



#plots for troubleshooting 
def depthplot(dropg, drop, accNameg): #Velocity and develeration vs. penetration depth 
    fig, (ax1) = plt.subplots(1)
    ax1.plot(dropg[accNameg], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "solid") #marker = 11
    ax1.plot(drop["Velocity (m/s)"], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "dashed")
    ax1.set(xlabel="Deceleration (g) and Velocity (m/s)", ylabel="Penetration Depth (cm)")
    ax1.invert_yaxis()
    ax1.legend(["Deceleration (g)", "Velocity (m/s)"])
    ax1.set_title("BD file "+fileNum)
    plt.show()

def peakplot(): # Plot showing peak deceleration
    peakplot = plt.plot(x)
    peakplot = plt.plot(peaks, x[peaks], "x")
    plt.show()

def qdynplot(drop, qdyn): #Plot showing dynamic bearing capacity vs depth
    #print(len(drop))
    #print(len(qdyn))
    #print((bctable.iloc[:,0]))
    #print(len(bctable.iloc[:,len(bctable.columns)-1]))
    fig, (ax1) = plt.subplots(1)

    if droptype == "a": #for partial saturation, only one qsbc curve
        ax1.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax1.plot(qsbc, drop["Penetration Depth (m)"]*100, label="QSBC") #plots QSBC
        ax1.set(xlabel="Bearing Capacity (kPa)")
        ax1.set_xlim(0,)
        ax1.invert_yaxis()
        ax1.legend(["Qdyn", "QSBC"])
        ax1.set_title("Bearing Capacity- "+fileNum+ " "+str(a))
        ax1.grid()

    else:
        ax1.plot(qdyn, drop["Penetration Depth (m)"]*100, label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax1.plot(bctable.iloc[:,0], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[0])) #plots QSBC with smallest srf
        ax1.plot(bctable.iloc[:,len(bctable.columns)-1], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[len(bctable.columns)-1])) #plots QSBC with largest srf
        for i in range(1,len(bctable.columns)-1): #plots qsbc for all other srfs
            ax1.plot(bctable.iloc[:,i], drop["Penetration Depth (m)"]*100, label=str(bctable.columns[i]), color = "k")
        ax1.set(xlabel="Bearing Capacity (kPa)", ylabel="Penetration Depth (cm)")
        ax1.set_xlim(0,)
        ax1.invert_yaxis()
        ax1.legend(["Qdyn", str(bctable.columns[0]), str(bctable.columns[len(bctable.columns)-1])])
        ax1.set_title("Bearing Capacity- "+fileNum+ " "+str(a))
    
    plt.show()

def tiltplot():
    fig, (ax1) = plt.subplots(1)
    plt.plot(time, gX55g, label="2g")
    plt.plot(time, gY55g, label="2g")
    ax1.legend()
    ax1.set(ylabel="Deceleration (g)")
    ax1.set(xlabel="Time (s)")
    ax1.set_title("BD file "+fileNum)


In [13]:
from matplotlib.font_manager import FontProperties
font = FontProperties()
font.set_name('Calibri')

def duck_dep_qsbc_comboplot(dropg, drop, dropt, accNameg, qdynt):
    font = {'fontname': 'Calibri'}
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(7, 7)
    ax1.plot(dropg[accNameg], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "solid") #marker = 11
    ax1.plot(drop["Velocity (m/s)"], drop["Penetration Depth (m)"]*100, color = "k", linestyle = "dashed")
    #ax1.plot(drop["X55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "b", linestyle = "dashed")
    #ax1.plot(drop["Y55g (m/s^2)"], drop["Penetration Depth (m)"]*100, color = "r", linestyle = "dashed")
    ax1.set(xlabel="Deceleration (g) and Velocity (m/s)", ylabel="Penetration Depth (cm)")
    ax1.set_xlim(left = 0)
    ax1.set_ylim(0,dep.max()*100+.25)
    ax1.invert_yaxis()
    ax1.legend(["Deceleration (g)", "Velocity (m/s)"])
    ax1.set_title("BD file "+fileNum)
    ax1.grid()

    
    if droptype == "a": #for partial saturation, only one qsbc curve
        y = dropt["Penetration Depth (m)"]*100
        ax2.plot(qdynt[10:-50], y[10:-50], label="Qdyn") #color = "k", marker = 11 #Plots Qdyn, chops off first 40 readings
        ax2.plot(qsbc[10:-50], y[10:-50], label="QSBC") #plots QSBC; chops off last 10 points which go off towards infinity (can adjust as needed)
        ax2.set(xlabel="Bearing Capacity (kPa)",)
        ax2.set_xlim(0,qdyn.max()+10)
        ax2.set_ylim(0,dep.max()*100+.25)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", "QSBC"], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()

    else: #for saturated drops, use a range of strain-rate factors
        y = list(dropt["Penetration Depth (m)"]*100)
        ax2.plot(qdynt[10:-20], y[10:-20], label="Qdyn") #color = "k", marker = 11 #Plots Qdyn
        ax2.plot(bctablet.iloc[10:-20,0], y[10:-20], label=str(bctablet.columns[0])) #plots QSBC with smallest srf
        ax2.plot(bctablet.iloc[10:-20,len(bctable.columns)-1], y[10:-20], label=str(bctable.columns[len(bctable.columns)-1])) #plots QSBC with largest srf
        for i in range(1,len(bctablet.columns)-1): #plots qsbc for all other srfs
            ax2.plot(bctablet.iloc[10:-20,i], y[10:-20], label=str(bctablet.columns[i]), color = '.6')
            #bctablet.iloc[:,i].pop()
        ax2.set(xlabel="Bearing Capacity (kPa)")
        ax2.set_xlim(0,)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", str(bctablet.columns[0]), str(bctablet.columns[len(bctablet.columns)-1])], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()

    #fig.subplots_adjust(bottom=.1, left = .1)
    plotName = fileNum+" "+str(a)+" BC"
    #plt.savefig(plotPath / plotName)
    plt.show()

In [10]:
#Exporting Functions
def exporttoexcel():
    with pd.ExcelWriter(outputPath, mode="a", if_sheet_exists='replace') as writer:
        output_table.to_excel(writer, sheet_name = fileNum, index=False)

def troubleshooting_export():
        with pd.ExcelWriter(troubleshootingPath, mode="a", if_sheet_exists = 'new') as writer:
            drop1.to_excel(writer, sheet_name = str(a), index = False)
            qdyntable.to_excel(writer, sheet_name = "qdyn", index = False)


Bring in data and calibrate for first sample

In [14]:
fileNum= '014F' # write the bin file number you want to analyze (do not include 'bLog' or '.bin')
a = 3 #what drop do you want to analyze? USER INPUT
a = a-1

#Set the penetrometer mass and length
masslength(tiptype)

# READ BD DATA IN
data_array = []  # creates an empty array for us to fill with bd data
fileName = 'bLog'+fileNum+".bin"
# print(fileName)
newPath = binFilepath / fileName
#print(newPath)
file = open(newPath, 'rb')  # read file
element = file.read(3)  # create a byte list with each element having 3 bytes

while element:
    # Convert to signed integer before adding to data array
    iVAl = int.from_bytes(element, byteorder='big', signed=True)
    data_array.append(iVAl)  # adds the reshaped data from the bd file to the data frame
    element = file.read(3)

np_array = np.array(data_array)  # create numpy array from the list
np_array = np.reshape(np_array, (-1, 10))  # convert the 1d array to 2d array with 10 cols

#print(np_array.shape)
# print(np_array)

df = pd.DataFrame(np_array) # Creates a Dataframe in pandas from the bd data
df.columns = ['Count', 'no clue', 'g2g', 'g18g', 'g50g', 'ppm', 'g200g', 'gX55g', 'gY55g', 'g250g'] # names columns
# print(dfCal)

# APPLY CALIBRATION FACTORS
if BD == 3:  # calibration factors from March 2023
    g2g = (df['g2g']-33570.1)/1614577.9 #- offset# accelerometers are in g
    g18g = (df['g18g']+13495)/163387.2 #- offset
    g50g = (df['g50g']-238817.4)/63779.5 #- offset
    ppm = ((df['ppm']-139040.1)/20705) * 6.89475729 # converts to kPa
    g200g = (df['g200g'] -262332.4)/38888.7 #- offset
    gX55g = (df['gX55g']-70406)/59754.3
    gY55g = (df['gY55g']-69421.1)/141871.5
    g250g = (df['g250g']-39077.4)/13746.9 #- offset

if BD == 2: # calibration factors from March 2023
    g2g = (df['g2g']+36597.6)/1637627.3 #- offset# accelerometers are in g
    g18g = (df['g18g']-26185.3)/160297.2 #- offset
    g50g = (df['g50g']-212256.9)/63968.7 #- offset
    ppm = ((df['ppm']+55518.9)/18981.7) * 6.89475729 # converts to kPa
    g200g = (df['g200g']-175499.4)/30583.8 #- offset
    gX55g = (df['gX55g']-53629.9)/68590.9
    gY55g = (df['gY55g']-43694.3)/68280.3
    g250g = (df['g250g']-39619.9)/13545.8 #- offset

if BD == 1: # calibration factors from March 2023
    g2g = (df['g2g']-43727.6)/1625064 #- offset # accelerometers are in g
    g18g = (df['g18g']-45085.6)/160925.7 #- offset
    g50g = (df['g50g']-173493.4)/63944.6 #- offset
    ppm = ((df['ppm']+31776.1)/20679.7) * 6.89475729 # this is kPa
    g200g = (df['g200g'] -731734.3)/32192.4  #- offset
    gX55g = (df['gX55g'] -68837.9)/52137.3
    gY55g = (df['gY55g']-68015.9)/28074.9
    g250g = (df['g250g']+10580)/13688.7 #- offset

time = (df['Count']-df['Count'].iloc[0]+1)/2000 # gives time in s
count = df["Count"]-df['Count'].iloc[0]

# make a new dataframe of the calibrated values in units of g
dfCalg = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCalg = dfCalg.T
dfCalg.columns = ['Count', 'Time (s)', '2g (g)', '18g (g)', '50g (g)', '200g (g)', '250g (g)', 'X55g (g)', 'Y55g (g)', 'Pore Pressure (kPa)']  # names columns
#print(dfCalg)

#make a new dataframe of the calibrated values in units of m/s^2
dfCal = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCal = dfCal.T
dfCal.columns = ['Count','Time (s)', '2g (m/s^2)', '18g (m/s^2)', '50g (m/s^2)', '200g (m/s^2)', '250g (m/s^2)', 'X55g (m/s^2)', 'Y55g (m/s^2)', 'Pore Pressure (kPa)']  # names columns
dfCal['2g (m/s^2)'] = dfCal['2g (m/s^2)'] * 9.80665
dfCal['18g (m/s^2)'] = dfCal['18g (m/s^2)'] * 9.80665
dfCal['50g (m/s^2)'] = dfCal['50g (m/s^2)'] * 9.80665
dfCal['200g (m/s^2)'] = dfCal['200g (m/s^2)'] * 9.80665
dfCal['250g (m/s^2)'] = dfCal['250g (m/s^2)'] * 9.80665
dfCal['X55g (m/s^2)'] = dfCal['X55g (m/s^2)'] * 9.80665
dfCal['Y55g (m/s^2)'] = dfCal['Y55g (m/s^2)'] * 9.80665
#print(dfCal)

#Locate the drops
x = np.array(g250g)  # what accelerometer to get the peaks from - use 250 because it will never be maxed out
max250 = max(g250g)

peaks, _ = find_peaks(x, height = 2, distance=1000, prominence=3)  # finds the largest peaks more than 2g spaced at least 10000 counts apart

peaksArray = np.array(peaks)  # prints a list of the count where the peaks occur
#peaksArray = filter(filterpeak, peaksArray)

#print(peaksArray)
q = (peaksArray.shape) #gives number of peaks
nDrops = int(q[0]) #number of drops in the file
#print(nDrops)

# For each drop, find the start and end points and integrate to solve for velocity and acceleration
#peakplot()

peakplot()
#overviewplot()

peak = int(peaksArray[a])  # count at the ath drop
print("peak index:" , peak)
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop1 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop1g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop1 = pd.DataFrame(drop1)  # makes dataframe including all data within the start and end points of the drop
drop1g = pd.DataFrame(drop1g)
dg = drop1g 
d = drop1
accPick(dg, d)  # chooses what accelerometer to use
acc1 = acc
acc1g = accg
acc1Name = accName
acc1Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peak = int(peaksArray[a])  # count at the ath drop
print("peak index:" , peak)
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop1 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop1g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop1 = pd.DataFrame(drop1)  # makes dataframe including all data within the start and end points of the drop
drop1g = pd.DataFrame(drop1g)
dg = drop1g 
d = drop1
accPick(dg, d)  # chooses what accelerometer to use
acc1 = acc
acc1g = accg
acc1Name = accName
acc1Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peakpick()

finddropend(a) #automatically find the end of the drop
print("penend: ", penend)
offset = acc1g[penstart] #sets the offset to be the acceleration at the moment of impact
print("offset: ", offset)
#print(drop)
dg = dfCalg[penstart:penend] #chops the file to only include penetration 
d = dfCal[penstart:penend]
acc1g = accg[penstart:penend]
acc1g = acc1g-offset
acc1 = acc[penstart:penend]
offsetms = offset * 9.80665
acc1 = acc1-offsetms


drop1 = d
drop1g = dg
integration(d, acc1) #double integration to solve for velocity and penetration depth
drop1 = d
areafind() #area calculations for bearing capacity calculations
qdynfun(acc1) #calculation of dynamic bearing capacity
qsbcfun(acc1) #calculation of quasi-static bearing capacities
qsbc1 = qsbc
qdyn1 = qdyn
integplot(drop1,acc1Name)
#qdynplot(drop1, qdyn1)
print("Max penetration depth is: ", maxdep)
print("Pre-truncation d1", drop1)
drop1 = pd.DataFrame(drop1)
drop1g = pd.DataFrame(drop1g)
acc1 = pd.DataFrame(acc1)
qdyn1 = pd.DataFrame(qdyn1)

drop1t = drop1 #.truncate(before=trunc_index) #this truncates each df such that only depths below cone height are considered
drop1gt = drop1g #.truncate(before=trunc_index)
acc1t = acc1 #.truncate(before=trunc_index)
acc1t = np.array(acc1t)
acc1gt = acc1g #.truncate(before=trunc_index)
acc1gt = np.array(acc1gt)
drop1t = drop1 #truncate(before=trunc_index)
qdyn1t = qdyn1#.truncate(before=trunc_index)

duck_dep_qsbc_comboplot(drop1g, drop1, drop1t, acc1Nameg, qdyn1t)




peak index: 52569
peak index: 52569
start of penetration:  52522
penend:  52595
offset:  -0.11577380812803753
[[0.00000000e+00 2.20000000e-01 1.52053084e-05]
 [1.99912229e-01 3.30017922e-01 4.00037594e-05]
 [3.99495817e-01 4.39854983e-01 8.21087376e-05]
 [5.98674146e-01 5.49469017e-01 1.40272516e-04]
 [7.97335481e-01 6.58798533e-01 2.14064942e-04]
 [9.95317945e-01 7.67754445e-01 3.03190190e-04]
 [1.19237563e+00 8.76201423e-01 4.07310090e-04]
 [1.38814051e+00 9.83936932e-01 5.25952310e-04]
 [1.58212064e+00 1.09069023e+00 6.58451743e-04]
 [1.77374864e+00 1.19614909e+00 8.03939924e-04]
 [1.96245662e+00 1.30000096e+00 9.61384172e-04]
 [2.14774379e+00 1.40197026e+00 1.12965623e-03]
 [2.32921072e+00 1.50183716e+00 1.30759779e-03]
 [2.50655550e+00 1.59943551e+00 1.49405868e-03]
 [2.67954773e+00 1.69463852e+00 1.68790633e-03]
 [2.84800107e+00 1.78734364e+00 1.88802091e-03]
 [3.01175882e+00 1.87746462e+00 2.09329509e-03]
 [3.17068923e+00 1.96492897e+00 2.30264029e-03]
 [3.32467829e+00 2.0496739

C:\Users\elise\AppData\Local\Temp\ipykernel_6600\2952692391.py:296: RuntimeWarning: divide by zero encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.


Max penetration depth is:  0.0567
Pre-truncation d1          Count  Time (s)  2g (m/s^2)  18g (m/s^2)  50g (m/s^2)  200g (m/s^2)  \
52522  52522.0   26.2615    0.123339    -0.090929    -1.135353      7.583530   
52523  52523.0   26.2620    0.589063     0.345147    -0.932839      8.081497   
52524  52524.0   26.2625    0.702590     0.445602    -0.669309      8.076366   
52525  52525.0   26.2630    0.983377     0.670002    -0.908923      8.360782   
52526  52526.0   26.2635    1.305609     1.057197    -0.393516      8.817706   
...        ...       ...         ...          ...          ...           ...   
52590  52590.0   26.2955   18.296934    18.860279    25.248705     26.429967   
52591  52591.0   26.2960   15.236179    15.091169    19.092327     22.685433   
52592  52592.0   26.2965   12.507849    12.346727    14.744169     19.967296   
52593  52593.0   26.2970   10.511706    10.234559    11.719331     17.858070   
52594  52594.0   26.2975    8.852879     8.612671     9.858375     1

Trial 2

In [30]:
fileNum= '0182' # write the bin file number you want to analyze (do not include 'bLog' or '.bin')
a = 4 #what drop do you want to analyze? USER INPUT
a = a-1

#Set the penetrometer mass and length
masslength(tiptype)

# READ BD DATA IN
data_array = []  # creates an empty array for us to fill with bd data
fileName = 'bLog'+fileNum+".bin"
# print(fileName)
newPath = binFilepath / fileName
#print(newPath)
file = open(newPath, 'rb')  # read file
element = file.read(3)  # create a byte list with each element having 3 bytes

while element:
    # Convert to signed integer before adding to data array
    iVAl = int.from_bytes(element, byteorder='big', signed=True)
    data_array.append(iVAl)  # adds the reshaped data from the bd file to the data frame
    element = file.read(3)

np_array = np.array(data_array)  # create numpy array from the list
np_array = np.reshape(np_array, (-1, 10))  # convert the 1d array to 2d array with 10 cols

#print(np_array.shape)
# print(np_array)

df = pd.DataFrame(np_array) # Creates a Dataframe in pandas from the bd data
df.columns = ['Count', 'no clue', 'g2g', 'g18g', 'g50g', 'ppm', 'g200g', 'gX55g', 'gY55g', 'g250g'] # names columns
# print(dfCal)

# APPLY CALIBRATION FACTORS
if BD == 3:  # calibration factors from March 2023
    g2g = (df['g2g']-33570.1)/1614577.9 #- offset# accelerometers are in g
    g18g = (df['g18g']+13495)/163387.2 #- offset
    g50g = (df['g50g']-238817.4)/63779.5 #- offset
    ppm = ((df['ppm']-139040.1)/20705) * 6.89475729 # converts to kPa
    g200g = (df['g200g'] -262332.4)/38888.7 #- offset
    gX55g = (df['gX55g']-70406)/59754.3
    gY55g = (df['gY55g']-69421.1)/141871.5
    g250g = (df['g250g']-39077.4)/13746.9 #- offset

if BD == 2: # calibration factors from March 2023
    g2g = (df['g2g']+36597.6)/1637627.3 #- offset# accelerometers are in g
    g18g = (df['g18g']-26185.3)/160297.2 #- offset
    g50g = (df['g50g']-212256.9)/63968.7 #- offset
    ppm = ((df['ppm']+55518.9)/18981.7) * 6.89475729 # converts to kPa
    g200g = (df['g200g']-175499.4)/30583.8 #- offset
    gX55g = (df['gX55g']-53629.9)/68590.9
    gY55g = (df['gY55g']-43694.3)/68280.3
    g250g = (df['g250g']-39619.9)/13545.8 #- offset

if BD == 1: # calibration factors from March 2023
    g2g = (df['g2g']-43727.6)/1625064 #- offset # accelerometers are in g
    g18g = (df['g18g']-45085.6)/160925.7 #- offset
    g50g = (df['g50g']-173493.4)/63944.6 #- offset
    ppm = ((df['ppm']+31776.1)/20679.7) * 6.89475729 # this is kPa
    g200g = (df['g200g'] -731734.3)/32192.4  #- offset
    gX55g = (df['gX55g'] -68837.9)/52137.3
    gY55g = (df['gY55g']-68015.9)/28074.9
    g250g = (df['g250g']+10580)/13688.7 #- offset

time = (df['Count']-df['Count'].iloc[0]+1)/2000 # gives time in s
count = df["Count"]-df['Count'].iloc[0]

# make a new dataframe of the calibrated values in units of g
dfCalg = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCalg = dfCalg.T
dfCalg.columns = ['Count', 'Time (s)', '2g (g)', '18g (g)', '50g (g)', '200g (g)', '250g (g)', 'X55g (g)', 'Y55g (g)', 'Pore Pressure (kPa)']  # names columns
#print(dfCalg)

#make a new dataframe of the calibrated values in units of m/s^2
dfCal = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCal = dfCal.T
dfCal.columns = ['Count','Time (s)', '2g (m/s^2)', '18g (m/s^2)', '50g (m/s^2)', '200g (m/s^2)', '250g (m/s^2)', 'X55g (m/s^2)', 'Y55g (m/s^2)', 'Pore Pressure (kPa)']  # names columns
dfCal['2g (m/s^2)'] = dfCal['2g (m/s^2)'] * 9.80665
dfCal['18g (m/s^2)'] = dfCal['18g (m/s^2)'] * 9.80665
dfCal['50g (m/s^2)'] = dfCal['50g (m/s^2)'] * 9.80665
dfCal['200g (m/s^2)'] = dfCal['200g (m/s^2)'] * 9.80665
dfCal['250g (m/s^2)'] = dfCal['250g (m/s^2)'] * 9.80665
dfCal['X55g (m/s^2)'] = dfCal['X55g (m/s^2)'] * 9.80665
dfCal['Y55g (m/s^2)'] = dfCal['Y55g (m/s^2)'] * 9.80665
#print(dfCal)

#Locate the drops
x = np.array(g250g)  # what accelerometer to get the peaks from - use 250 because it will never be maxed out
max250 = max(g250g)

peaks, _ = find_peaks(x, height = 2, distance=1000, prominence=3)  # finds the largest peaks more than 2g spaced at least 10000 counts apart

peaksArray = np.array(peaks)  # prints a list of the count where the peaks occur
#peaksArray = filter(filterpeak, peaksArray)

#print(peaksArray)
q = (peaksArray.shape) #gives number of peaks
nDrops = int(q[0]) #number of drops in the file
#print(nDrops)

# For each drop, find the start and end points and integrate to solve for velocity and acceleration
#peakplot()

peakplot()
#overviewplot()

peak = int(peaksArray[a])  # count at the ath drop
print("peak index:" , peak)
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop2 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop2g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop2 = pd.DataFrame(drop2)  # makes dataframe including all data within the start and end points of the drop
drop2g = pd.DataFrame(drop2g)
dg = drop2g 
d = drop2
accPick(dg, d)  # chooses what accelerometer to use
acc2 = acc
acc2g = accg
acc2Name = accName
acc2Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peak = int(peaksArray[a])  # count at the ath drop
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop2 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop2g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop2 = pd.DataFrame(drop2)  # makes dataframe including all data within the start and end points of the drop
drop2g = pd.DataFrame(drop2g)
dg = drop2g 
d = drop2
accPick(dg, d)  # chooses what accelerometer to use
acc2 = acc
acc2g = accg
acc2Name = accName
acc2Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peakpick()

finddropend(a) #automatically find the end of the drop
offset = acc2g[penstart] #sets the offset to be the acceleration at the moment of impact
#print(drop)
dg = dfCalg[penstart:penend] #chops the file to only include penetration 
d = dfCal[penstart:penend]
acc2g = accg[penstart:penend]
acc2g = acc2g-offset
acc2 = acc[penstart:penend]
offsetms = offset * 9.80665
acc2 = acc2-offsetms


drop2 = d
drop2g = dg
integration(d, acc2) #double integration to solve for velocity and penetration depth
drop2 = d
areafind() #area calculations for bearing capacity calculations
qdynfun(acc2) #calculation of dynamic bearing capacity
qsbcfun(acc2) #calculation of quasi-static bearing capacities
qsbc2 = qsbc
qdyn2 = qdyn
integplot(drop2,acc2Name)
#qdynplot(drop1, qdyn1)
print("Max penetration depth is: ", maxdep)
print("Pre-truncation d1", drop2)
drop2 = pd.DataFrame(drop2)
drop2g = pd.DataFrame(drop2g)
acc2 = pd.DataFrame(acc2)
qdyn2 = pd.DataFrame(qdyn2)

drop2t = drop2 #.truncate(before=trunc_index) #this truncates each df such that only depths below cone height are considered
drop2gt = drop2g #.truncate(before=trunc_index)
acc2t = acc2 #.truncate(before=trunc_index)
acc2t = np.array(acc2t)
acc2gt = acc2g #.truncate(before=trunc_index)
acc2gt = np.array(acc2gt)
drop2t = drop2 #truncate(before=trunc_index)
qdyn2t = qdyn2#.truncate(before=trunc_index)

duck_dep_qsbc_comboplot(drop2g, drop2, drop2t, acc2Nameg, qdyn2t)



peak index: 53576
start of penetration:  53537
[[0.00000000e+00 2.20000000e-01 1.52053084e-05]
 [2.67703840e-01 3.67325756e-01 5.24517085e-05]
 [5.34974050e-01 5.14412871e-01 1.19940456e-04]
 [8.01651552e-01 6.61173801e-01 2.15842204e-04]
 [1.06747736e+00 8.07466015e-01 3.39534301e-04]
 [1.33206241e+00 9.53075403e-01 4.90418300e-04]
 [1.59488199e+00 1.09771320e+00 6.67689633e-04]
 [1.85533038e+00 1.24104606e+00 8.70280151e-04]
 [2.11281431e+00 1.38274748e+00 1.09689750e-03]
 [2.36681864e+00 1.52253397e+00 1.34610152e-03]
 [2.61693180e+00 1.66017902e+00 1.61637741e-03]
 [2.86285098e+00 1.79551601e+00 1.90619918e-03]
 [3.10435842e+00 1.92842507e+00 2.21405621e-03]
 [3.34128051e+00 2.05881067e+00 2.53844011e-03]
 [3.57346740e+00 2.18659034e+00 2.87783793e-03]
 [3.80079196e+00 2.31169412e+00 3.23074425e-03]
 [4.02314557e+00 2.43406223e+00 3.59566688e-03]
 [4.24042806e+00 2.55363955e+00 3.97111934e-03]
 [4.45254383e+00 2.67037346e+00 4.35561946e-03]
 [4.65940481e+00 2.78421550e+00 4.7476988

C:\Users\elise\AppData\Local\Temp\ipykernel_6600\2952692391.py:296: RuntimeWarning: divide by zero encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.


Max penetration depth is:  0.0835
Pre-truncation d1          Count  Time (s)  2g (m/s^2)  18g (m/s^2)  50g (m/s^2)  200g (m/s^2)  \
53537  53537.0   26.7690   -0.021884    -0.325301    -1.561538     12.106918   
53538  53538.0   26.7695   -0.002523    -0.299851    -1.297549     12.115255   
53539  53539.0   26.7700    0.408521     0.062016    -0.986648     12.454822   
53540  53540.0   26.7705    1.107617     0.777492    -0.579473     13.174357   
53541  53541.0   26.7710    2.166898     1.849819     0.150253     14.221275   
...        ...       ...         ...          ...          ...           ...   
53608  53608.0   26.8045   19.580556    29.610954    46.478073     42.543516   
53609  53609.0   26.8050   18.903300    20.800537    31.219584     33.568885   
53610  53610.0   26.8055   15.805783    15.601148    21.108273     28.315380   
53611  53611.0   26.8060   12.700919    12.385391    15.151038     25.038352   
53612  53612.0   26.8065   10.586530    10.319596    11.724236     2

Trial 3

In [27]:
fileNum= '01BE' # write the bin file number you want to analyze (do not include 'bLog' or '.bin')
a = 1 #what drop do you want to analyze? USER INPUT
a = a-1

#Set the penetrometer mass and length
masslength(tiptype)

# READ BD DATA IN
data_array = []  # creates an empty array for us to fill with bd data
fileName = 'bLog'+fileNum+".bin"
# print(fileName)
newPath = binFilepath / fileName
#print(newPath)
file = open(newPath, 'rb')  # read file
element = file.read(3)  # create a byte list with each element having 3 bytes

while element:
    # Convert to signed integer before adding to data array
    iVAl = int.from_bytes(element, byteorder='big', signed=True)
    data_array.append(iVAl)  # adds the reshaped data from the bd file to the data frame
    element = file.read(3)

np_array = np.array(data_array)  # create numpy array from the list
np_array = np.reshape(np_array, (-1, 10))  # convert the 1d array to 2d array with 10 cols

#print(np_array.shape)
# print(np_array)

df = pd.DataFrame(np_array) # Creates a Dataframe in pandas from the bd data
df.columns = ['Count', 'no clue', 'g2g', 'g18g', 'g50g', 'ppm', 'g200g', 'gX55g', 'gY55g', 'g250g'] # names columns
# print(dfCal)

# APPLY CALIBRATION FACTORS
if BD == 3:  # calibration factors from March 2023
    g2g = (df['g2g']-33570.1)/1614577.9 #- offset# accelerometers are in g
    g18g = (df['g18g']+13495)/163387.2 #- offset
    g50g = (df['g50g']-238817.4)/63779.5 #- offset
    ppm = ((df['ppm']-139040.1)/20705) * 6.89475729 # converts to kPa
    g200g = (df['g200g'] -262332.4)/38888.7 #- offset
    gX55g = (df['gX55g']-70406)/59754.3
    gY55g = (df['gY55g']-69421.1)/141871.5
    g250g = (df['g250g']-39077.4)/13746.9 #- offset

if BD == 2: # calibration factors from March 2023
    g2g = (df['g2g']+36597.6)/1637627.3 #- offset# accelerometers are in g
    g18g = (df['g18g']-26185.3)/160297.2 #- offset
    g50g = (df['g50g']-212256.9)/63968.7 #- offset
    ppm = ((df['ppm']+55518.9)/18981.7) * 6.89475729 # converts to kPa
    g200g = (df['g200g']-175499.4)/30583.8 #- offset
    gX55g = (df['gX55g']-53629.9)/68590.9
    gY55g = (df['gY55g']-43694.3)/68280.3
    g250g = (df['g250g']-39619.9)/13545.8 #- offset

if BD == 1: # calibration factors from March 2023
    g2g = (df['g2g']-43727.6)/1625064 #- offset # accelerometers are in g
    g18g = (df['g18g']-45085.6)/160925.7 #- offset
    g50g = (df['g50g']-173493.4)/63944.6 #- offset
    ppm = ((df['ppm']+31776.1)/20679.7) * 6.89475729 # this is kPa
    g200g = (df['g200g'] -731734.3)/32192.4  #- offset
    gX55g = (df['gX55g'] -68837.9)/52137.3
    gY55g = (df['gY55g']-68015.9)/28074.9
    g250g = (df['g250g']+10580)/13688.7 #- offset

time = (df['Count']-df['Count'].iloc[0]+1)/2000 # gives time in s
count = df["Count"]-df['Count'].iloc[0]

# make a new dataframe of the calibrated values in units of g
dfCalg = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCalg = dfCalg.T
dfCalg.columns = ['Count', 'Time (s)', '2g (g)', '18g (g)', '50g (g)', '200g (g)', '250g (g)', 'X55g (g)', 'Y55g (g)', 'Pore Pressure (kPa)']  # names columns
#print(dfCalg)

#make a new dataframe of the calibrated values in units of m/s^2
dfCal = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCal = dfCal.T
dfCal.columns = ['Count','Time (s)', '2g (m/s^2)', '18g (m/s^2)', '50g (m/s^2)', '200g (m/s^2)', '250g (m/s^2)', 'X55g (m/s^2)', 'Y55g (m/s^2)', 'Pore Pressure (kPa)']  # names columns
dfCal['2g (m/s^2)'] = dfCal['2g (m/s^2)'] * 9.80665
dfCal['18g (m/s^2)'] = dfCal['18g (m/s^2)'] * 9.80665
dfCal['50g (m/s^2)'] = dfCal['50g (m/s^2)'] * 9.80665
dfCal['200g (m/s^2)'] = dfCal['200g (m/s^2)'] * 9.80665
dfCal['250g (m/s^2)'] = dfCal['250g (m/s^2)'] * 9.80665
dfCal['X55g (m/s^2)'] = dfCal['X55g (m/s^2)'] * 9.80665
dfCal['Y55g (m/s^2)'] = dfCal['Y55g (m/s^2)'] * 9.80665
#print(dfCal)

#Locate the drops
x = np.array(g250g)  # what accelerometer to get the peaks from - use 250 because it will never be maxed out
max250 = max(g250g)

peaks, _ = find_peaks(x, height = 2, distance=1000, prominence=3)  # finds the largest peaks more than 2g spaced at least 10000 counts apart

peaksArray = np.array(peaks)  # prints a list of the count where the peaks occur
#peaksArray = filter(filterpeak, peaksArray)

#print(peaksArray)
q = (peaksArray.shape) #gives number of peaks
nDrops = int(q[0]) #number of drops in the file
#print(nDrops)

# For each drop, find the start and end points and integrate to solve for velocity and acceleration
#peakplot()

peakplot()
#overviewplot()

peak = int(peaksArray[a])  # count at the ath drop
print("peak index:" , peak)
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop3 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop3g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop3 = pd.DataFrame(drop3)  # makes dataframe including all data within the start and end points of the drop
drop3g = pd.DataFrame(drop3g)
dg = drop3g 
d = drop3
accPick(dg, d)  # chooses what accelerometer to use
acc3 = acc
acc3g = accg
acc3Name = accName
acc3Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peak = int(peaksArray[a])  # count at the ath drop
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop3 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop3g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop3 = pd.DataFrame(drop3)  # makes dataframe including all data within the start and end points of the drop
drop3g = pd.DataFrame(drop3g)
dg = drop3g 
d = drop3
accPick(dg, d)  # chooses what accelerometer to use
acc3 = acc
acc3g = accg
acc3Name = accName
acc3Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peakpick()

finddropend(a) #automatically find the end of the drop
offset = acc3g[penstart] #sets the offset to be the acceleration at the moment of impact
#print(drop)
dg = dfCalg[penstart:penend] #chops the file to only include penetration 
d = dfCal[penstart:penend]
acc3g = accg[penstart:penend]
acc3g = acc3g-offset
acc3 = acc[penstart:penend]
offsetms = offset * 9.80665
acc3 = acc3-offsetms


drop3 = d
drop3g = dg
integration(d, acc3) #double integration to solve for velocity and penetration depth
drop3 = d
areafind() #area calculations for bearing capacity calculations
qdynfun(acc3) #calculation of dynamic bearing capacity
qsbcfun(acc3) #calculation of quasi-static bearing capacities
qsbc3 = qsbc
qdyn3 = qdyn
integplot(drop3,acc3Name)
#qdynplot(drop1, qdyn1)
drop3 = pd.DataFrame(drop3)
drop3g = pd.DataFrame(drop3g)
acc3 = pd.DataFrame(acc3)
qdyn3 = pd.DataFrame(qdyn3)

drop3t = drop3 #.truncate(before=trunc_index) #this truncates each df such that only depths below cone height are considered
drop3gt = drop3g #.truncate(before=trunc_index)
acc3t = acc3 #.truncate(before=trunc_index)
acc3t = np.array(acc3t)
acc3gt = acc3g #.truncate(before=trunc_index)
acc3gt = np.array(acc3gt)
drop3t = drop3 #truncate(before=trunc_index)
qdyn3t = qdyn3#.truncate(before=trunc_index)

duck_dep_qsbc_comboplot(drop3g, drop3, drop3t, acc3Nameg, qdyn3t)



peak index: 100949
start of penetration:  100907
[[0.00000000e+00 2.20000000e-01 1.52053084e-05]
 [2.66738636e-01 3.66794574e-01 5.22609137e-05]
 [5.33079768e-01 5.13370389e-01 1.19360672e-04]
 [7.98922656e-01 6.59672005e-01 2.14717646e-04]
 [1.06412463e+00 8.05620903e-01 3.37800002e-04]
 [1.32849110e+00 9.51110004e-01 4.88197433e-04]
 [1.59177893e+00 1.09600549e+00 6.65437420e-04]
 [1.85371491e+00 1.24015701e+00 8.68940983e-04]
 [2.11402188e+00 1.38341204e+00 1.09802198e-03]
 [2.37243880e+00 1.52562692e+00 1.35190351e-03]
 [2.62872662e+00 1.66667008e+00 1.62973283e-03]
 [2.88266070e+00 1.80641791e+00 1.93058442e-03]
 [3.13402669e+00 1.94475244e+00 2.25346318e-03]
 [3.38262020e+00 2.08156118e+00 2.59731105e-03]
 [3.62823619e+00 2.21673131e+00 2.96099767e-03]
 [3.87066541e+00 2.35014765e+00 3.34331656e-03]
 [4.10970849e+00 2.48170050e+00 3.74300979e-03]
 [4.34518551e+00 2.61129083e+00 4.15879256e-03]
 [4.57693287e+00 2.73882862e+00 4.58935892e-03]
 [4.80480250e+00 2.86423237e+00 5.03339

C:\Users\elise\AppData\Local\Temp\ipykernel_6600\2952692391.py:296: RuntimeWarning: divide by zero encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.


trial 4

In [46]:
fileNum= '01F8' # write the bin file number you want to analyze (do not include 'bLog' or '.bin')
a = 2 #what drop do you want to analyze? USER INPUT
a = a-1

#Set the penetrometer mass and length
masslength(tiptype)

# READ BD DATA IN
data_array = []  # creates an empty array for us to fill with bd data
fileName = 'bLog'+fileNum+".bin"
# print(fileName)
newPath = binFilepath / fileName
#print(newPath)
file = open(newPath, 'rb')  # read file
element = file.read(3)  # create a byte list with each element having 3 bytes

while element:
    # Convert to signed integer before adding to data array
    iVAl = int.from_bytes(element, byteorder='big', signed=True)
    data_array.append(iVAl)  # adds the reshaped data from the bd file to the data frame
    element = file.read(3)

np_array = np.array(data_array)  # create numpy array from the list
np_array = np.reshape(np_array, (-1, 10))  # convert the 1d array to 2d array with 10 cols

#print(np_array.shape)
# print(np_array)

df = pd.DataFrame(np_array) # Creates a Dataframe in pandas from the bd data
df.columns = ['Count', 'no clue', 'g2g', 'g18g', 'g50g', 'ppm', 'g200g', 'gX55g', 'gY55g', 'g250g'] # names columns
# print(dfCal)

# APPLY CALIBRATION FACTORS
if BD == 3:  # calibration factors from March 2023
    g2g = (df['g2g']-33570.1)/1614577.9 #- offset# accelerometers are in g
    g18g = (df['g18g']+13495)/163387.2 #- offset
    g50g = (df['g50g']-238817.4)/63779.5 #- offset
    ppm = ((df['ppm']-139040.1)/20705) * 6.89475729 # converts to kPa
    g200g = (df['g200g'] -262332.4)/38888.7 #- offset
    gX55g = (df['gX55g']-70406)/59754.3
    gY55g = (df['gY55g']-69421.1)/141871.5
    g250g = (df['g250g']-39077.4)/13746.9 #- offset

if BD == 2: # calibration factors from March 2023
    g2g = (df['g2g']+36597.6)/1637627.3 #- offset# accelerometers are in g
    g18g = (df['g18g']-26185.3)/160297.2 #- offset
    g50g = (df['g50g']-212256.9)/63968.7 #- offset
    ppm = ((df['ppm']+55518.9)/18981.7) * 6.89475729 # converts to kPa
    g200g = (df['g200g']-175499.4)/30583.8 #- offset
    gX55g = (df['gX55g']-53629.9)/68590.9
    gY55g = (df['gY55g']-43694.3)/68280.3
    g250g = (df['g250g']-39619.9)/13545.8 #- offset

if BD == 1: # calibration factors from March 2023
    g2g = (df['g2g']-43727.6)/1625064 #- offset # accelerometers are in g
    g18g = (df['g18g']-45085.6)/160925.7 #- offset
    g50g = (df['g50g']-173493.4)/63944.6 #- offset
    ppm = ((df['ppm']+31776.1)/20679.7) * 6.89475729 # this is kPa
    g200g = (df['g200g'] -731734.3)/32192.4  #- offset
    gX55g = (df['gX55g'] -68837.9)/52137.3
    gY55g = (df['gY55g']-68015.9)/28074.9
    g250g = (df['g250g']+10580)/13688.7 #- offset

time = (df['Count']-df['Count'].iloc[0]+1)/2000 # gives time in s
count = df["Count"]-df['Count'].iloc[0]

# make a new dataframe of the calibrated values in units of g
dfCalg = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCalg = dfCalg.T
dfCalg.columns = ['Count', 'Time (s)', '2g (g)', '18g (g)', '50g (g)', '200g (g)', '250g (g)', 'X55g (g)', 'Y55g (g)', 'Pore Pressure (kPa)']  # names columns
#print(dfCalg)

#make a new dataframe of the calibrated values in units of m/s^2
dfCal = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCal = dfCal.T
dfCal.columns = ['Count','Time (s)', '2g (m/s^2)', '18g (m/s^2)', '50g (m/s^2)', '200g (m/s^2)', '250g (m/s^2)', 'X55g (m/s^2)', 'Y55g (m/s^2)', 'Pore Pressure (kPa)']  # names columns
dfCal['2g (m/s^2)'] = dfCal['2g (m/s^2)'] * 9.80665
dfCal['18g (m/s^2)'] = dfCal['18g (m/s^2)'] * 9.80665
dfCal['50g (m/s^2)'] = dfCal['50g (m/s^2)'] * 9.80665
dfCal['200g (m/s^2)'] = dfCal['200g (m/s^2)'] * 9.80665
dfCal['250g (m/s^2)'] = dfCal['250g (m/s^2)'] * 9.80665
dfCal['X55g (m/s^2)'] = dfCal['X55g (m/s^2)'] * 9.80665
dfCal['Y55g (m/s^2)'] = dfCal['Y55g (m/s^2)'] * 9.80665
#print(dfCal)

#Locate the drops
x = np.array(g250g)  # what accelerometer to get the peaks from - use 250 because it will never be maxed out
max250 = max(g250g)

peaks, _ = find_peaks(x, height = 2, distance=1000, prominence=3)  # finds the largest peaks more than 2g spaced at least 10000 counts apart

peaksArray = np.array(peaks)  # prints a list of the count where the peaks occur
#peaksArray = filter(filterpeak, peaksArray)

#print(peaksArray)
q = (peaksArray.shape) #gives number of peaks
nDrops = int(q[0]) #number of drops in the file
#print(nDrops)

# For each drop, find the start and end points and integrate to solve for velocity and acceleration
#peakplot()

peakplot()
#overviewplot()

peak = int(peaksArray[a])  # count at the ath drop
print("peak index:" , peak)
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop4 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop4g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop4 = pd.DataFrame(drop4)  # makes dataframe including all data within the start and end points of the drop
drop4g = pd.DataFrame(drop4g)
dg = drop4g 
d = drop4
accPick(dg, d)  # chooses what accelerometer to use
acc4 = acc
acc4g = accg
acc4Name = accName
acc4Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peak = int(peaksArray[a])  # count at the ath drop
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop4 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop4g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop4 = pd.DataFrame(drop4)  # makes dataframe including all data within the start and end points of the drop
drop4g = pd.DataFrame(drop4g)
dg = drop4g 
d = drop4
accPick(dg, d)  # chooses what accelerometer to use
acc4 = acc
acc4g = accg
acc4Name = accName
acc4Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peakpick()

finddropend(a) #automatically find the end of the drop
offset = acc4g[penstart] #sets the offset to be the acceleration at the moment of impact
#print(drop)
dg = dfCalg[penstart:penend] #chops the file to only include penetration 
d = dfCal[penstart:penend]
acc4g = accg[penstart:penend]
acc4g = acc4g-offset
acc4 = acc[penstart:penend]
offsetms = offset * 9.80665
acc4 = acc4-offsetms


drop4 = d
drop4g = dg
integration(d, acc4) #double integration to solve for velocity and penetration depth
drop4 = d
areafind() #area calculations for bearing capacity calculations
qdynfun(acc4) #calculation of dynamic bearing capacity
qsbcfun(acc4) #calculation of quasi-static bearing capacities
qsbc4 = qsbc
qdyn4 = qdyn
integplot(drop4,acc4Name)
#qdynplot(drop1, qdyn1)
drop4 = pd.DataFrame(drop4)
drop4g = pd.DataFrame(drop4g)
acc4 = pd.DataFrame(acc4)
qdyn4 = pd.DataFrame(qdyn4)

drop4t = drop4 #.truncate(before=trunc_index) #this truncates each df such that only depths below cone height are considered
drop4gt = drop4g #.truncate(before=trunc_index)
acc4t = acc4 #.truncate(before=trunc_index)
acc4t = np.array(acc4t)
acc4gt = acc4g #.truncate(before=trunc_index)
acc4gt = np.array(acc4gt)
drop4t = drop4 #truncate(before=trunc_index)
qdyn4t = qdyn4#.truncate(before=trunc_index)

duck_dep_qsbc_comboplot(drop4g, drop4, drop4t, acc4Nameg, qdyn4t)

dep4 = dep



peak index: 101915
start of penetration:  101866
[[0.00000000e+00 2.20000000e-01 1.52053084e-05]
 [2.57757345e-01 3.61851890e-01 5.05041996e-05]
 [5.15175812e-01 5.03517284e-01 1.13951766e-04]
 [7.72205013e-01 6.44968454e-01 2.03863616e-04]
 [1.02878502e+00 7.86172420e-01 3.19790098e-04]
 [1.28483987e+00 9.27087375e-01 4.61459660e-04]
 [1.54027976e+00 1.06766390e+00 6.28614554e-04]
 [1.79500552e+00 1.20784741e+00 8.20972270e-04]
 [2.04891515e+00 1.34758178e+00 1.03821619e-03]
 [2.30191080e+00 1.48681316e+00 1.27999739e-03]
 [2.55389868e+00 1.62548993e+00 1.54593523e-03]
 [2.80478597e+00 1.76356102e+00 1.83561517e-03]
 [3.05448177e+00 1.90097639e+00 2.14859042e-03]
 [3.30290078e+00 2.03768911e+00 2.48438824e-03]
 [3.54996352e+00 2.17365542e+00 2.84251324e-03]
 [3.79558942e+00 2.30883100e+00 3.22243959e-03]
 [4.03968391e+00 2.44316380e+00 3.62358903e-03]
 [4.28213167e+00 2.57659034e+00 4.04531251e-03]
 [4.52280663e+00 2.70904126e+00 4.48689975e-03]
 [4.76158383e+00 2.84044779e+00 4.94759

C:\Users\elise\AppData\Local\Temp\ipykernel_6600\2952692391.py:296: RuntimeWarning: divide by zero encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.


trial 5

In [33]:
fileNum= '023B' # write the bin file number you want to analyze (do not include 'bLog' or '.bin')
a = 3 #what drop do you want to analyze? USER INPUT
a = a-1

#Set the penetrometer mass and length
masslength(tiptype)

# READ BD DATA IN
data_array = []  # creates an empty array for us to fill with bd data
fileName = 'bLog'+fileNum+".bin"
# print(fileName)
newPath = binFilepath / fileName
#print(newPath)
file = open(newPath, 'rb')  # read file
element = file.read(3)  # create a byte list with each element having 3 bytes

while element:
    # Convert to signed integer before adding to data array
    iVAl = int.from_bytes(element, byteorder='big', signed=True)
    data_array.append(iVAl)  # adds the reshaped data from the bd file to the data frame
    element = file.read(3)

np_array = np.array(data_array)  # create numpy array from the list
np_array = np.reshape(np_array, (-1, 10))  # convert the 1d array to 2d array with 10 cols

#print(np_array.shape)
# print(np_array)

df = pd.DataFrame(np_array) # Creates a Dataframe in pandas from the bd data
df.columns = ['Count', 'no clue', 'g2g', 'g18g', 'g50g', 'ppm', 'g200g', 'gX55g', 'gY55g', 'g250g'] # names columns
# print(dfCal)

# APPLY CALIBRATION FACTORS
if BD == 3:  # calibration factors from March 2023
    g2g = (df['g2g']-33570.1)/1614577.9 #- offset# accelerometers are in g
    g18g = (df['g18g']+13495)/163387.2 #- offset
    g50g = (df['g50g']-238817.4)/63779.5 #- offset
    ppm = ((df['ppm']-139040.1)/20705) * 6.89475729 # converts to kPa
    g200g = (df['g200g'] -262332.4)/38888.7 #- offset
    gX55g = (df['gX55g']-70406)/59754.3
    gY55g = (df['gY55g']-69421.1)/141871.5
    g250g = (df['g250g']-39077.4)/13746.9 #- offset

if BD == 2: # calibration factors from March 2023
    g2g = (df['g2g']+36597.6)/1637627.3 #- offset# accelerometers are in g
    g18g = (df['g18g']-26185.3)/160297.2 #- offset
    g50g = (df['g50g']-212256.9)/63968.7 #- offset
    ppm = ((df['ppm']+55518.9)/18981.7) * 6.89475729 # converts to kPa
    g200g = (df['g200g']-175499.4)/30583.8 #- offset
    gX55g = (df['gX55g']-53629.9)/68590.9
    gY55g = (df['gY55g']-43694.3)/68280.3
    g250g = (df['g250g']-39619.9)/13545.8 #- offset

if BD == 1: # calibration factors from March 2023
    g2g = (df['g2g']-43727.6)/1625064 #- offset # accelerometers are in g
    g18g = (df['g18g']-45085.6)/160925.7 #- offset
    g50g = (df['g50g']-173493.4)/63944.6 #- offset
    ppm = ((df['ppm']+31776.1)/20679.7) * 6.89475729 # this is kPa
    g200g = (df['g200g'] -731734.3)/32192.4  #- offset
    gX55g = (df['gX55g'] -68837.9)/52137.3
    gY55g = (df['gY55g']-68015.9)/28074.9
    g250g = (df['g250g']+10580)/13688.7 #- offset

time = (df['Count']-df['Count'].iloc[0]+1)/2000 # gives time in s
count = df["Count"]-df['Count'].iloc[0]

# make a new dataframe of the calibrated values in units of g
dfCalg = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCalg = dfCalg.T
dfCalg.columns = ['Count', 'Time (s)', '2g (g)', '18g (g)', '50g (g)', '200g (g)', '250g (g)', 'X55g (g)', 'Y55g (g)', 'Pore Pressure (kPa)']  # names columns
#print(dfCalg)

#make a new dataframe of the calibrated values in units of m/s^2
dfCal = pd.DataFrame([count, time, g2g, g18g, g50g, g200g, g250g, gX55g, gY55g, ppm])
dfCal = dfCal.T
dfCal.columns = ['Count','Time (s)', '2g (m/s^2)', '18g (m/s^2)', '50g (m/s^2)', '200g (m/s^2)', '250g (m/s^2)', 'X55g (m/s^2)', 'Y55g (m/s^2)', 'Pore Pressure (kPa)']  # names columns
dfCal['2g (m/s^2)'] = dfCal['2g (m/s^2)'] * 9.80665
dfCal['18g (m/s^2)'] = dfCal['18g (m/s^2)'] * 9.80665
dfCal['50g (m/s^2)'] = dfCal['50g (m/s^2)'] * 9.80665
dfCal['200g (m/s^2)'] = dfCal['200g (m/s^2)'] * 9.80665
dfCal['250g (m/s^2)'] = dfCal['250g (m/s^2)'] * 9.80665
dfCal['X55g (m/s^2)'] = dfCal['X55g (m/s^2)'] * 9.80665
dfCal['Y55g (m/s^2)'] = dfCal['Y55g (m/s^2)'] * 9.80665
#print(dfCal)

#Locate the drops
x = np.array(g250g)  # what accelerometer to get the peaks from - use 250 because it will never be maxed out
max250 = max(g250g)

peaks, _ = find_peaks(x, height = 2, distance=1000, prominence=3)  # finds the largest peaks more than 2g spaced at least 10000 counts apart

peaksArray = np.array(peaks)  # prints a list of the count where the peaks occur
#peaksArray = filter(filterpeak, peaksArray)

#print(peaksArray)
q = (peaksArray.shape) #gives number of peaks
nDrops = int(q[0]) #number of drops in the file
#print(nDrops)

# For each drop, find the start and end points and integrate to solve for velocity and acceleration
#peakplot()

peakplot()
#overviewplot()

peak = int(peaksArray[a])  # count at the ath drop
print("peak index:" , peak)
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop5 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop5g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop5 = pd.DataFrame(drop5)  # makes dataframe including all data within the start and end points of the drop
drop5g = pd.DataFrame(drop5g)
dg = drop5g 
d = drop5
accPick(dg, d)  # chooses what accelerometer to use
acc5 = acc
acc5g = accg
acc5Name = accName
acc5Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peak = int(peaksArray[a])  # count at the ath drop
dropstartend(peak) #zooms in the drop file to only consider 500 counts before and 1500 counts after the peak deceleration
#print(dropstart, dropend)

drop5 = dfCal[dropstart:dropend]  # start and end points of the drop in m/s^2
drop5g = dfCalg[dropstart:dropend]  # start and end points of the drop in g
drop5 = pd.DataFrame(drop5)  # makes dataframe including all data within the start and end points of the drop
drop5g = pd.DataFrame(drop5g)
dg = drop5g 
d = drop5
accPick(dg, d)  # chooses what accelerometer to use
acc5 = acc
acc5g = accg
acc5Name = accName
acc5Nameg = accNameg
#findchangepts() #automatically identifies the moment of penetration

peakpick()

finddropend(a) #automatically find the end of the drop
offset = acc5g[penstart] #sets the offset to be the acceleration at the moment of impact
#print(drop)
dg = dfCalg[penstart:penend] #chops the file to only include penetration 
d = dfCal[penstart:penend]
acc5g = accg[penstart:penend]
acc5g = acc5g-offset
acc5 = acc[penstart:penend]
offsetms = offset * 9.80665
acc5 = acc5-offsetms


drop5 = d
drop5g = dg
integration(d, acc5) #double integration to solve for velocity and penetration depth
drop5 = d
areafind() #area calculations for bearing capacity calculations
qdynfun(acc5) #calculation of dynamic bearing capacity
qsbcfun(acc5) #calculation of quasi-static bearing capacities
qsbc5 = qsbc
qdyn5 = qdyn
integplot(drop5,acc5Name)
#qdynplot(drop1, qdyn1)
drop5 = pd.DataFrame(drop5)
drop5g = pd.DataFrame(drop5g)
acc5 = pd.DataFrame(acc5)
qdyn5 = pd.DataFrame(qdyn5)

drop5t = drop5 #.truncate(before=trunc_index) #this truncates each df such that only depths below cone height are considered
drop5gt = drop5g #.truncate(before=trunc_index)
acc5t = acc5 #.truncate(before=trunc_index)
acc5t = np.array(acc5t)
acc5gt = acc5g #.truncate(before=trunc_index)
acc5gt = np.array(acc5gt)
drop5t = drop5 #truncate(before=trunc_index)
qdyn5t = qdyn5#.truncate(before=trunc_index)

duck_dep_qsbc_comboplot(drop5g, drop5, drop5t, acc5Nameg, qdyn5t)



peak index: 60350
start of penetration:  60304
[[0.00000000e+00 2.20000000e-01 1.52053084e-05]
 [2.72580056e-01 3.70009289e-01 5.34215405e-05]
 [5.44830716e-01 5.19837301e-01 1.22980473e-04]
 [8.16682537e-01 6.69445820e-01 2.22089324e-04]
 [1.08804672e+00 8.18785978e-01 3.50271767e-04]
 [1.35880239e+00 9.67791249e-01 5.07207124e-04]
 [1.62878662e+00 1.11637197e+00 6.92545557e-04]
 [1.89779297e+00 1.26441454e+00 9.05849722e-04]
 [2.16557905e+00 1.41178556e+00 1.14656888e-03]
 [2.43188175e+00 1.55834022e+00 1.41403305e-03]
 [2.69643192e+00 1.70393041e+00 1.70745882e-03]
 [2.95896941e+00 1.84841297e+00 2.02596612e-03]
 [3.21925780e+00 1.99165777e+00 2.36860452e-03]
 [3.47708930e+00 2.13355047e+00 2.73437453e-03]
 [3.73228287e+00 2.27399144e+00 3.12224111e-03]
 [3.98468320e+00 2.41289519e+00 3.53114727e-03]
 [4.23415352e+00 2.55018647e+00 3.96001599e-03]
 [4.48056341e+00 2.68579351e+00 4.40773816e-03]
 [4.72378579e+00 2.81964635e+00 4.87317132e-03]
 [4.96370190e+00 2.95167965e+00 5.3551531

C:\Users\elise\AppData\Local\Temp\ipykernel_6600\2952692391.py:296: RuntimeWarning: divide by zero encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.
C:\Users\elise\AppData\Local\Temp\ipykernel_6600\2952692391.py:296: RuntimeWarning: invalid value encountered in log10
  srcv = np.log10(vel/0.02) #Velocity portion of the strain rate correction.


In [1]:
def combined_duck_dep_qsbc_comboplot():
    fig, (ax1, ax2) = plt.subplots(1,2)
    fig.set_size_inches(7, 7)
    fig.set_dpi(300)
    ax1.plot(drop1g[acc1Nameg], drop1["Penetration Depth (m)"]*100, linestyle = "solid", color = "orange") #marker = 11
    ax1.plot(drop1["Velocity (m/s)"], drop1["Penetration Depth (m)"]*100, color = "orange", linestyle = "dashed")

    ax1.plot(drop2g[acc2Nameg], drop2["Penetration Depth (m)"]*100, linestyle = "solid", color = "orangered") #marker = 11
    ax1.plot(drop2["Velocity (m/s)"], drop2["Penetration Depth (m)"]*100, linestyle = "dashed", color = "orangered")

    ax1.plot(drop3g[acc3Nameg], drop3["Penetration Depth (m)"]*100, linestyle = "solid", color = "darkviolet") #marker = 11
    ax1.plot(drop3["Velocity (m/s)"], drop3["Penetration Depth (m)"]*100, linestyle = "dashed", color = "darkviolet")

    ax1.plot(drop4g[acc4Nameg], drop4["Penetration Depth (m)"]*100, linestyle = "solid", color = "dodgerblue") #marker = 11
    ax1.plot(drop4["Velocity (m/s)"], drop4["Penetration Depth (m)"]*100, linestyle = "dashed", color = "dodgerblue")

    ax1.plot(drop5g[acc5Nameg], drop5["Penetration Depth (m)"]*100, linestyle = "solid", color = "limegreen") #marker = 11
    ax1.plot(drop5["Velocity (m/s)"], drop5["Penetration Depth (m)"]*100, linestyle = "dashed", color = "limegreen")


    ax1.set(xlabel="Deceleration (g) and Velocity (m/s)", ylabel="Penetration Depth (cm)")
    ax1.set_xlim(left = 0)
    ax1.set_ylim(0,dep4.max()*100+.25)
    ax1.invert_yaxis()
    ax1.legend(["Deceleration (g)", "Velocity (m/s)"])
    ax1.set_title("Station Q3")
    ax1.grid()

    
    if droptype == "a": #for partial saturation, only one qsbc curve
        y1 = drop1t["Penetration Depth (m)"]*100
        ax2.plot(qdyn1t[5:-50], y1[5:-50], label="Qdyn", linestyle = "dashed", color = "orange") #color = "k", marker = 11 #Plots Qdyn, chops off first 40 readings
        ax2.plot(qsbc1[5:-50], y1[5:-50], label="QSBC", color = "orange") #plots QSBC; chops off last 10 points which go off towards infinity (can adjust as needed)
        y2 = drop2t["Penetration Depth (m)"]*100
        ax2.plot(qdyn2t[5:-47], y2[5:-47], label="Qdyn", linestyle = "dashed", color = "orangered") #color = "k", marker = 11 #Plots Qdyn, chops off first 40 readings
        ax2.plot(qsbc2[5:-47], y2[5:-47], label="QSBC", color = "orangered") #plots QSBC; chops off last 10 points which go off towards infinity (can adjust as needed)
        y3 = drop3t["Penetration Depth (m)"]*100
        ax2.plot(qdyn3t[5:-50], y3[5:-50], label="Qdyn", linestyle = "dashed", color = "darkviolet") #color = "k", marker = 11 #Plots Qdyn, chops off first 40 readings
        ax2.plot(qsbc3[5:-50], y3[5:-50], label="QSBC", color = "darkviolet") #plots QSBC; chops off last 10 points which go off towards infinity (can adjust as needed)
        y4 = drop4t["Penetration Depth (m)"]*100
        ax2.plot(qdyn4t[5:-50], y4[5:-50], label="Qdyn", linestyle = "dashed", color = "dodgerblue") #color = "k", marker = 11 #Plots Qdyn, chops off first 40 readings
        ax2.plot(qsbc4[5:-50], y4[5:-50], label="QSBC", color = "dodgerblue") #plots QSBC; chops off last 10 points which go off towards infinity (can adjust as needed)
        y5 = drop5t["Penetration Depth (m)"]*100
        ax2.plot(qdyn5t[5:-50], y5[5:-50], label="Qdyn", linestyle = "dashed", color = "limegreen") #color = "k", marker = 11 #Plots Qdyn, chops off first 40 readings
        ax2.plot(qsbc5[5:-50], y5[5:-50], label="QSBC", color = "limegreen") #plots QSBC; chops off last 10 points which go off towards infinity (can adjust as needed)
        
        
        ax2.set(xlabel="Bearing Capacity (kPa)",)
        ax2.set_xlim(left = 0)
        ax2.set_ylim(0,dep4.max()*100+.25)
        ax2.invert_yaxis()
        ax2.legend(["Qdyn", "QSBC"], loc = "upper right")
        ax2.set_title("Drop #"+str(a))
        ax2.grid()

    

combined_duck_dep_qsbc_comboplot()

plt.show()


NameError: name 'plt' is not defined